In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"joshuaojo","key":"8c6cdd7731bf63d92f7d4bc5130c5b39"}'}

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nikitarom/planets-dataset
!mkdir planets_dataset
!unzip planets-dataset -d planets_dataset

In [40]:
from tqdm import tqdm
from keras.preprocessing import image
import cv2 

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow

In [42]:
# train_tags= pd.read_csv("/content/planets_dataset/planet/planet/train_classes.csv")
train_tags.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in train_tags['tags'].values])))

In [ ]:
label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}
label_map

In [43]:
x_train=[]
y_train=[]
x_test=[]

## Image Preprocessing 

In [44]:
# Custom Data Augmentation

for f, tags in tqdm(train_tags.values, miniters=400):
        img = cv2.imread('/content/planets_dataset/planet/planet/train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        img = cv2.resize(img, (64,64))
        flipped_img=cv2.flip(img,1)
        rows,cols,channel = img.shape
        # regular
        # x_train.append(img)
        # y_train.append(targets)
        
        # flipped
        x_train.append(flipped_img)
        y_train.append(targets)

        # rotated
        for rotate_degree in [90]:
            M = cv2.getRotationMatrix2D((cols/2,rows/2),rotate_degree,1)
            dst = cv2.warpAffine(img,M,(cols,rows))
            x_train.append(dst)
            y_train.append(targets)
            
            dst = cv2.warpAffine(flipped_img,M,(cols,rows))
            x_train.append(dst)
            y_train.append(targets)

100%|██████████| 40479/40479 [00:58<00:00, 694.68it/s]


In [46]:
y_train = np.array(y_train, np.uint8)

In [45]:
x_train = np.array(x_train, np.float32)

In [47]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

In [48]:
print(x_train.shape)
print(y_train.shape)

(97149, 64, 64, 3)
(97149, 17)


In [ ]:
kf = KFold(5, shuffle=True, random_state=42)

In [49]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout,Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

### **Base model**

In [ ]:
model = Sequential()
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu", input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 256)       0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 128)       295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        7

In [50]:
from keras import backend as K
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)
    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[fbeta])

In [ ]:
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 20, batch_size = 128)

Epoch 1/20
253/253 [==============================] - 11s 43ms/step - loss: 0.3082 - fbeta: 0.5813 - val_loss: 0.2439 - val_fbeta: 0.6088
Epoch 2/20
253/253 [==============================] - 11s 42ms/step - loss: 0.2407 - fbeta: 0.6037 - val_loss: 0.2427 - val_fbeta: 0.6074
Epoch 3/20
253/253 [==============================] - 10s 41ms/step - loss: 0.2258 - fbeta: 0.6237 - val_loss: 0.2107 - val_fbeta: 0.6587
Epoch 4/20
253/253 [==============================] - 10s 41ms/step - loss: 0.2147 - fbeta: 0.6479 - val_loss: 0.2028 - val_fbeta: 0.6708
Epoch 5/20
253/253 [==============================] - 11s 42ms/step - loss: 0.2072 - fbeta: 0.6588 - val_loss: 0.2037 - val_fbeta: 0.7021
Epoch 6/20
253/253 [==============================] - 11s 42ms/step - loss: 0.1998 - fbeta: 0.6747 - val_loss: 0.1833 - val_fbeta: 0.7043
Epoch 7/20
253/253 [==============================] - 10s 41ms/step - loss: 0.1916 - fbeta: 0.6967 - val_loss: 0.1758 - val_fbeta: 0.7279
Epoch 8/20
253/253 [==============

In [51]:
df_test = pd.read_csv("/content/planets_dataset/planet/planet/sample_submission.csv")
df_test

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water
...,...,...
61186,file_9995,primary clear agriculture road water
61187,file_9996,primary clear agriculture road water
61188,file_9997,primary clear agriculture road water
61189,file_9998,primary clear agriculture road water


In [52]:
x_test = []
for f, tags in tqdm(df_test[:40669].values, miniters=1000):
    img = cv2.imread('/content/planets_dataset/planet/planet/test-jpg/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (64, 64)))

100%|██████████| 40669/40669 [01:04<00:00, 628.08it/s]


In [53]:
for f, tags in tqdm(df_test[40669:].values, miniters=1000):
    img = cv2.imread('/content/planets_dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (64, 64)))

100%|██████████| 20522/20522 [00:30<00:00, 665.69it/s]


In [54]:
x_test  = np.array(x_test, np.float32)/255.

In [55]:
x_test.shape

(61191, 64, 64, 3)

In [ ]:
base_model_pred=model.predict(x_test,batch_size=128,verbose=2)

479/479 - 5s


In [ ]:
result = pd.DataFrame(base_model_pred, columns = labels)

In [ ]:
preds=[]
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [02:45<00:00, 369.33it/s]


In [ ]:
df_test['tags']= pd.DataFrame(preds)
df_test.to_csv('submit1', index=False)

### Using a pre-trained model || Transfer Learning  (VGG16)

In [56]:
from tensorflow.keras.applications import VGG16

In [57]:
conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(64,64,3))

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [58]:
v_model=Sequential()
v_model.add(BatchNormalization(input_shape=64,64))
v_model.add(conv_base)
v_model.add(Flatten())
v_model.add(Dense(4096,activation='relu'))
v_model.add(Dropout(0.5))
v_model.add(Dense(4096,activation='relu'))
v_model.add(Dropout(0.5))
v_model.add(Dense(17, activation='sigmoid'))

In [ ]:
v_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              8392704   
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                6

In [ ]:
conv_base.trainable=False

In [ ]:
len(v_model.trainable_weights)

32

In [59]:
v_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[fbeta])

In [ ]:
v_model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs = 10, batch_size = 64)

In [65]:
vgg16_pred=v_model.predict(x_test,batch_size=128,verbose=2)

479/479 - 11s


In [70]:
vgg16_result = pd.DataFrame(vgg16_pred, columns = labels)

In [71]:
pred=[]
for i in tqdm(range(vgg16_result.shape[0]), miniters=1000):
    a = vgg16_result.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    pred.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [04:02<00:00, 252.24it/s]


In [72]:
predictions_vg16=pd.DataFrame(pred)

In [73]:
df_test['tags']=predictions_vg16
df_test.to_csv('submission_final', index=False)